In [1]:
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import numpy as np

In [2]:
links = ['http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/0f69cde4-567f-468b-8bc1-fc861ac26590/download/bdg_serie_iscritti.csv',
        'http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/3098c012-08de-4085-b532-66c00e72a6cf/download/bdg_serie_laureati.csv',
        'http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/74fc14c9-571f-4181-888e-ef17e014793e/download/bdg_serie_dottorandi.csv',
        'http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/155998a7-c183-4eb3-b8cd-ca4d7db72b64/download/bdg_serie_dottori.csv']

In [3]:
def leggi_csv(link):
    df = pd.read_csv(link,encoding = "ISO-8859-1",sep=';')
    df = df.rename(columns={
        df.columns[-1]:'Numero'
    })
    categoria = os.path.split(link)[-1][:-4].split('_')[-1]
    df['categoria'] = [categoria]*len(df)
    return df

In [4]:
dfs = [leggi_csv(x) for x in links]

In [5]:
stud = pd.concat(dfs,join='inner')

In [6]:
stud = stud[stud['CorsoTIPO']!='Vecchio Ordinamento'] #leviamo i vecchi ordinamenti

In [7]:
stud['ANNO'].unique() #uniformiamo tutto sull'anno a sinistra in quelli con formato x/x+1

array(['2021/2022', '2020/2021', '2019/2020', '2018/2019', '2017/2018',
       '2016/2017', '2015/2016', '2014/2015', '2013/2014', '2012/2013',
       2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012],
      dtype=object)

In [8]:
diz = dict(zip(stud['ANNO'].unique()[0:10],stud['ANNO'].unique()[10:])) #mappiamo vecchi valori a nuovi

In [9]:
stud['ANNO'].loc[:] = stud['ANNO'].apply(lambda x: diz.get(x, x)).astype(int)

In [10]:
prof = pd.read_csv('http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/92f2008d-958f-4e9c-ae5c-7a3dd418cd57/download/bdg_serie_academic_staff_ambito.csv',encoding = "ISO-8859-1", delimiter=';')

In [11]:
stud.columns

Index(['ANNO', 'AteneoCOD', 'AteneoNOME', 'AteneoREGIONE', 'AteneoAREAGEO',
       'CorsoTIPO', 'COD_FoET2013', 'DESC_FoET2013', 'Genere', 'Numero',
       'categoria'],
      dtype='object')

In [12]:
prof = prof.rename(columns={
    'CODICE_ATENEO':'AteneoCOD',
    'NOME_ATENEO':'AteneoNOME',
    'REGIONE':'AteneoREGIONE',
    'AREA_GEO': 'AteneoAREAGEO',
    'GENERE':'Genere',
    'N_AcStaff':'Numero',
    'GRADE':'categoria'
})

In [13]:
decod = pd.read_csv('http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/17b34084-9c01-4e90-9260-982fcb982e6a/download/cod_grade.csv', encoding = "ISO-8859-1", delimiter=';')

In [14]:
decod_grade = dict(zip(decod['GRADE'],decod['Qualifica']))

In [15]:
decod_grade['C'] = 'Ricercatore'

In [16]:
prof['categoria'] = prof['categoria'].apply(lambda x: decod_grade[x])

In [17]:
#troviamo per materie prof, se è stem o no!
materie_prof = pd.read_csv('http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/688a0ad5-a4f7-469d-af3d-f7448aa5ae30/download/cod_ford.csv',encoding = "ISO-8859-1", delimiter=';')
decoder_prof = dict(zip(materie_prof['FoRD'],materie_prof['Area STEM']))

In [18]:
prof['STEM'] = prof['FoRD'].apply(lambda x: decoder_prof[x])

In [19]:
#ora stessa cosa per stud
materie_stud = pd.read_csv('http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/3f52db2f-24ce-4605-8e51-5618cc4ff4e3/download/cod_foet2013.csv',encoding = "ISO-8859-1", delimiter=';')
decoder_stud = dict(zip(materie_stud['ISCED_F_1dgt'],materie_stud['Area STEM']))

In [20]:
stud.sample()

,ANNO,AteneoCOD,AteneoNOME,AteneoREGIONE,AteneoAREAGEO,CorsoTIPO,COD_FoET2013,DESC_FoET2013,Genere,Numero,categoria
14210,2017,05802,"Roma - Università degli studi di ""Tor Vergata""",Lazio,CENTRO,Laurea Magistrale,7,"Engineering, manufacturing and construction",M,734,iscritti


In [21]:
stud['STEM'] = stud['COD_FoET2013'].apply(lambda x: decoder_stud[x])

In [22]:
df = pd.concat([stud,prof],join='inner') #adesso sappiamo se sono STEM o no!

In [23]:
df['STEM'] = df['STEM'].replace('No',False).replace('Sì',True).astype(bool) #trasformiamo in booleani

In [24]:
def conto(df, verbose=False, relative=False, flourish=False): 
    '''
    
    
    
    big funzione che serve a fare i conti
    
    
    '''
    
    df = df.groupby(['categoria','Genere']).agg({
        'Numero':pd.Series.sum
    }).reset_index()
    
    diz = {
            'iscritti':0,
            'laureati':1,
            'dottorandi':2,
            'dottori':3,
            'Assegnista di Ricerca':4,
            'Ricercatore':5,
            'Prof. Associato':6,
            'Prof. Ordinario':7
        }
    
    def ordina(colonna): #funzione di supporto solo per averli in ordine 
        colonna = colonna.apply(lambda x:diz[x])
        return colonna
    
    df = df.sort_values(by=['categoria'],key=ordina)
    
    if flourish:
        return df
    
    M= []
    F=[]
    colons = []

    for cat in diz:
        try:
            M.append(df[(df['categoria']==cat)&(df['Genere']=='M')]['Numero'].iloc[0])
        except IndexError as e:
            if verbose:
                print(f'M - {cat} ---> {e}')
            M.append(0)
        
        try:
            F.append(df[(df['categoria']==cat)&(df['Genere']=='F')]['Numero'].iloc[0])
            
        except IndexError as e:
            if verbose:
                print(f'F - {cat} ---> {e}')
            F.append(0)
            
        colons.append(cat)
    
    ndf = pd.DataFrame([M,F],columns=colons,index=['M','F'])
    if relative:
        ndf = ndf.apply(lambda x: x/x.sum()*100)
        
    return ndf

pd.DataFrame.conto = conto

In [25]:
def filtro(df, nome, stem=None, anno=2021): 
    '''
    
    funzione per filtrare per università, STEM (False,True,None), anno
    
    
    '''
    
    if stem is not None:
        df = df[df['STEM']==stem]
        
    return df[(df['AteneoNOME']==nome)&(df['ANNO']==anno)]
pd.DataFrame.filtro = filtro

In [26]:
diz = {
            'iscritti':0,
            'laureati':1,
            'dottorandi':2,
            'dottori':3,
            'Assegnista di Ricerca':4,
            'Ricercatore':5,
            'Prof. Associato':6,
            'Prof. Ordinario':7
        }
    
def ordina(colonna, reverse=False): #funzione di supporto solo per averli in ordine 
        if reverse:
            colonna = colonna.apply(lambda x:(diz[x]*-1))
        else:
            colonna = colonna.apply(lambda x:diz[x])
        return colonna

In [27]:
M = []
F = []
CAT = []
ndf = df.filtro('TOTALE ATENEI').groupby(['categoria','Genere']).agg({
        'Numero':pd.Series.sum
    })

In [28]:
lista_df = []

for università in df['AteneoNOME'].unique():

    M = []
    F = []
    CAT = []
    ndf = df.filtro(università).groupby(['categoria','Genere']).agg({
            'Numero':pd.Series.sum
        })
    
    if ('iscritti' in ndf.index.get_level_values('categoria')) and (ndf.loc['iscritti'].sum()[0] > 500):
    
        for cat in df['categoria'].unique():
            try:
                femmine_ass = ndf.loc[cat,'F'][0]
            except KeyError:
                femmine_ass = 0

            try:
                maschi_ass = ndf.loc[cat,'M'][0]
            except KeyError:
                maschi_ass = 0

            tot = femmine_ass+maschi_ass
            if tot>0:

                femmine = femmine_ass/tot*100
                maschi = maschi_ass/tot*100
            else: 
                femmine = 0
                maschi = 0

            CAT.append(cat) 
            M.append(maschi)
            F.append(femmine)

        df_uni = pd.DataFrame({
        'cat':CAT,
        'F':F,
        'M':M,
        'uni':[università]*len(M)
        }).sort_values(by='cat',key=lambda x: ordina(x,reverse=True))

        lista_df.append(df_uni)

In [29]:
nn = pd.concat(lista_df)

In [286]:
nn['F'] = nn['F'].apply(lambda x: x*(-1))

In [287]:
diz_cat = {'Assegnista di Ricerca':'Assegnisti/e di Ricerca',
          'Prof. Associato':'Prof. Associati/e',
          'Prof. Ordinario':'Prof. Ordinari/e',
          'Ricercatore':'Ricercatori/Ricercatrici',
          'dottorandi':'Dottorandi/e',
           'dottori':'Dottori/Dottoresse',
          'iscritti':'Iscritti/e',
          'laureati':'Laureati/e'}

In [288]:
nn['cat'] = nn['cat'].apply(lambda x: diz_cat[x])

In [289]:
nn.to_csv('/Users/sava/Desktop/task/Nucleo uni/Università genere/dati_flourish_STEM.csv',index=False)